In [41]:
from transformers import BertTokenizerFast, BertForSequenceClassification
import tensorflow as tf
import pandas as pd
from transformers import pipeline

In [18]:
# Loading data
train_set = pd.read_csv("../data/twitter_training.csv",names=["Tweet Id","Entity","Sentiment","Tweet Content"])
train_set

,Tweet Id,Entity,Sentiment,Tweet Content
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
...,...,...,...,...
74677,9200,Nvidia,Positive,Just realized that the Windows partition of my...
74678,9200,Nvidia,Positive,Just realized that my Mac window partition is ...
74679,9200,Nvidia,Positive,Just realized the windows partition of my Mac ...
74680,9200,Nvidia,Positive,Just realized between the windows partition of...


In [19]:
# Cleaning data
## Remove rows with missing values
hasnan=train_set.isna().any(axis=1)
droppedcount = 0
for i in hasnan.index:
    if hasnan[i]:
        print("dropped",i)
        droppedcount += 1
        train_set=train_set.drop(i)
print("Total Dropped:",droppedcount)

dropped 61
dropped 553
dropped 589
dropped 745
dropped 1105
dropped 1106
dropped 2295
dropped 2296
dropped 2297
dropped 2359
dropped 2413
dropped 2473
dropped 2474
dropped 2929
dropped 2983
dropped 2997
dropped 2998
dropped 2999
dropped 3061
dropped 3097
dropped 3243
dropped 3244
dropped 3245
dropped 3337
dropped 3457
dropped 3697
dropped 3835
dropped 3836
dropped 3939
dropped 3940
dropped 3941
dropped 3967
dropped 4233
dropped 4234
dropped 4235
dropped 4237
dropped 4413
dropped 4414
dropped 4415
dropped 4755
dropped 4756
dropped 4757
dropped 4869
dropped 4870
dropped 4871
dropped 4899
dropped 4900
dropped 4901
dropped 5421
dropped 5422
dropped 5423
dropped 5521
dropped 5559
dropped 5560
dropped 5561
dropped 5731
dropped 5973
dropped 5974
dropped 5975
dropped 6015
dropped 6016
dropped 6017
dropped 6067
dropped 6068
dropped 6081
dropped 6082
dropped 6083
dropped 6585
dropped 6586
dropped 6587
dropped 6699
dropped 6700
dropped 6701
dropped 6861
dropped 6862
dropped 6863
dropped 6903
drop

In [20]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [24]:
max_len = 128
X_train_enc = tokenizer.batch_encode_plus(train_set["Tweet Content"],padding=True,truncation=True,max_length=max_len,return_tensors="pt")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [28]:
k = 100
print('Training Comments -->>',train_set["Tweet Content"][k])
print('\nInput Ids -->>\n',X_train_enc['input_ids'][k])
print('\nDecoded Ids -->>\n',tokenizer.decode(X_train_enc['input_ids'][k]))
print('\nAttention Mask -->>\n',X_train_enc['attention_mask'][k])
print('\nLabels -->>',train_set["Sentiment"][k])

Training Comments -->> Deep Grounded almost looked pretty cool even despite the borderlands fifth tier unfunny writing format until ultimately it became yet another survival crafting mobile game. I really can ’ t wait for this shitty trend starting to... die

Input Ids -->>
 tensor([  101, 16764,  2471,  2001,  3492,  4658,  2130,  2750,  1996,  2327,
         7563,  4895, 11263, 10695,  2100,  3015,  2127,  2057,  2150,  2664,
         2178, 15703,  7477,  2075,  2208,  1012,  1045,  5667,  2064,  1521,
         1056,  3524,  2006,  2023, 28543,  9874,  2000,  3280,   102,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0, 

In [29]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=3)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [38]:
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile()

In [55]:
model(input_ids = X_train_enc['input_ids'], labels=train_set["Sentiment"])

RuntimeError: [enforce fail at alloc_cpu.cpp:124] err == 0. DefaultCPUAllocator: can't allocate memory: you tried to allocate 29096411136 bytes. Error code 12 (Cannot allocate memory)

In [44]:
model_p = pipeline(task='sentiment-analysis',model='bert-base-uncased',tokenizer='bert-base-uncased',framework='pt')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cpu


In [54]:
k = 7777
print(model_p.predict(train_set["Tweet Content"][k]))
print(train_set["Sentiment"][k])
print(train_set["Tweet Content"][k])

[{'label': 'LABEL_0', 'score': 0.6496325135231018}]
Negative
I'll just say it - being nominated for the LGBTQ + game of the year is essentially worse than being nominated for the LGBTQ + movie of the year for Rise of Skywalker. At least in Rise of Skywalker, if you slow down and zoom in, you can see homosexual shit when you pinch yourself.
